In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import vtk
import math
from vtk.util import numpy_support

from params import *

In [2]:
def vti_reader(file_path):
    reader = vtk.vtkXMLImageDataReader()
    reader.SetFileName(file_path)
    reader.Update()
    data = reader.GetOutput()
    
    # vtk res
    dataset = pd.DataFrame(np.array([numpy_support.vtk_to_numpy(data.GetPointData().GetArray(i)) for i in range(data.GetPointData().GetNumberOfArrays())])).transpose()
    
    dataset.columns=[data.GetPointData().GetArrayName(i) for i in range(data.GetPointData().GetNumberOfArrays())]
    
    # vtk coord
    dimensions = data.GetDimensions()
    spacing=data.GetSpacing()
    origin=data.GetOrigin()
    
    coordinates=[]
    
    for k in range(dimensions[2]):
        for j in range(dimensions[1]):
            for i in range(dimensions[0]):
                x = origin[0] + i * spacing[0]
                y = origin[1] + j * spacing[1]
                z = origin[2] + k * spacing[2]
                coordinates.append((x,y,z))
    dt_coordinates = pd.DataFrame(coordinates, columns=['x','y','z'])
    
    # result coord merge
    return pd.merge(dt_coordinates, dataset, left_index=True, right_index=True)

In [3]:
path_saved = '/home/hero/Git/ai_modulus/coating_modulus/outputs/'
doe_name = 'twophase_coat_v7_2_1_tr'

In [4]:
truth_df = pd.read_csv('2023_pinn_datamodel_xy.csv')
doe_truth = truth_df[truth_df['DOE']=='DOE0'][['contact1x','contact2x','contact3x']]

In [5]:
dirnames = os.listdir(path_saved+doe_name)
# 'windows'를 포함하는 이름만 선택
dir_windows = [name for name in dirnames if 'window' in name.lower()]
dir_windows.sort()

In [6]:
result_df = pd.DataFrame()

In [7]:
for i,window_name in enumerate(dir_windows):
    file_path = path_saved + doe_name + '/' + window_name + '/inferencers/window_000t.vti'
    vti_data = vti_reader(file_path)
    vti_slurry = vti_data[vti_data['alpha']==0.0].dropna()

    left_max_y = vti_slurry[vti_slurry['x']<0.0]['y'].max()
    right_max_y = vti_slurry[vti_slurry['x']>Lf]['y'].max()

    pnt_left_up = vti_slurry[(vti_slurry['x']<0.0) & (vti_slurry['y']==left_max_y)]['x'].min() * L_ref
    pnt_left_dn = vti_slurry[vti_slurry['y']==0.0]['x'].min() * L_ref
    pnt_right_up = vti_slurry[(vti_slurry['x']>Lf) & (vti_slurry['y'] == right_max_y)]['x'].iloc[0] * L_ref
    #pnt_right_dn = vti_slurry[vti_slurry['y']==0.0]['x'].min() * L_ref
    
    pred_array = np.array([pnt_left_up,pnt_left_dn,pnt_right_up])
    error = np.abs(( pred_array - doe_truth.values) / doe_truth.values) * 100
    

    df1 = pd.DataFrame([[doe_name,window_name] + list(np.hstack([pred_array, error.squeeze()]))],index=[i],
                       columns=['DOE','window','pred_contact1x','pred_contact2x','pred_contact3x','err1x','err2x','err3x'])
    
    result_df = pd.concat([result_df,df1],axis=0)

In [8]:
result_df

,DOE,window,pred_contact1x,pred_contact2x,pred_contact3x,err1x,err2x,err3x
0,twophase_coat_v7_2_2,window_0000,-0.000339,-0.000311,0.001508,3.859717,95.612341,3.068448
1,twophase_coat_v7_2_2,window_0001,-0.000394,-0.000394,0.001508,20.767113,147.610558,3.068448
2,twophase_coat_v7_2_2,window_0002,-0.000476,-0.000449,0.001508,46.128206,182.276036,3.068448
3,twophase_coat_v7_2_2,window_0003,-0.000504,-0.000476,0.001508,54.581904,199.608775,3.068448
4,twophase_coat_v7_2_2,window_0004,-0.000531,-0.000531,0.001508,63.035602,234.274253,3.068448
5,twophase_coat_v7_2_2,window_0005,-0.000559,-0.000587,0.001508,71.489300,268.939732,3.068448
6,twophase_coat_v7_2_2,window_0006,-0.000614,-0.000642,0.001508,88.396696,303.605210,3.068448
7,twophase_coat_v7_2_2,window_0007,-0.000642,-0.000669,0.001535,96.850394,320.937949,1.296853
8,twophase_coat_v7_2_2,window_0008,-0.000669,-0.000697,0.001535,105.304092,338.270688,1.296853
9,twophase_coat_v7_2_2,window_0009,-0.000669,-0.000724,0.001535,105.304092,355.603427,1.296853


In [9]:
doe_truth

,contact1x,contact2x,contact3x
0,-0.000326,-0.000159,0.001556
